In [1]:
# https://github.com/ryuikaneko/exact_diagonalization/blob/master/testing/191108_ed_sz_conserved_1d_heisenberg_J1J2_chain_spin_corr_numba.py

In [2]:
import numpy as np
#import scipy.linalg
import scipy.sparse
import scipy.sparse.linalg
from numba import jit
import time

In [3]:
@jit(nopython=True)
def snoob(x):
    next = 0
    if(x>0):
        smallest = x & -(x)
        ripple = x + smallest
        ones = x ^ ripple
        ones = (ones >> 2) // smallest
        next = ripple | ones
    return next

In [4]:
## https://stackoverflow.com/questions/26560726/python-binomial-coefficient
@jit(nopython=True)
def binomial(n,r):
    p = 1
    for i in range(1,min(r,n-r) + 1):
        p *= n
        p //= i
        n -= 1
    return p

In [5]:
@jit(nopython=True)
def init_parameters(N,Sz):
    Nup = N//2 + Sz
    Nhilbert = binomial(N,Nup)
    ihfbit = 1 << (N//2)
    irght = ihfbit-1
    ilft = ((1<<N)-1) ^ irght
    iup = (1<<(N-Nup))-1
    return Nup, Nhilbert, ihfbit, irght, ilft, iup

In [6]:
@jit(nopython=True)
def make_list(N,Nup,Nhilbert,ihfbit,irght,ilft,iup):
    list_1 = np.zeros(Nhilbert,dtype=np.int64)
    list_ja = np.zeros(ihfbit,dtype=np.int64)
    list_jb = np.zeros(ihfbit,dtype=np.int64)
    ii = iup
    ja = 0
    jb = 0
    ia_old = ii & irght
    ib_old = (ii & ilft) // ihfbit
    list_1[0] = ii
    list_ja[ia_old] = ja
    list_jb[ib_old] = jb
    ii = snoob(ii)
    for i in range(1,Nhilbert):
        ia = ii & irght
        ib = (ii & ilft) // ihfbit
        if (ib == ib_old):
            ja += 1
        else:
            jb += ja+1
            ja = 0
        list_1[i] = ii
        list_ja[ia] = ja
        list_jb[ib] = jb
        ia_old = ia
        ib_old = ib
        ii = snoob(ii)
    return list_1, list_ja, list_jb

In [7]:
@jit(nopython=True)
def get_ja_plus_jb(ii,irght,ilft,ihfbit,list_ja,list_jb):
    ia = ii & irght
    ib = (ii & ilft) // ihfbit
    ja = list_ja[ia]
    jb = list_jb[ib]
    return ja+jb

In [8]:
#def make_hamiltonian(Jxx,Jzz,list_isite1,list_isite2,N,Nint,Nhilbert,irght,ilft,ihfbit,list_1,list_ja,list_jb):
@jit(nopython=True)
def make_hamiltonian_child(Jxx,Jzz,list_isite1,list_isite2,N,Nint,Nhilbert,irght,ilft,ihfbit,list_1,list_ja,list_jb):
#    listki = np.zeros((Nint+1)*Nhilbert,dtype=np.int64)
    loc = np.zeros((Nint+1)*Nhilbert,dtype=np.int64)
    elemnt = np.zeros((Nint+1)*Nhilbert,dtype=np.float64)
    listki = np.array([i for k in range(Nint+1) for i in range(Nhilbert)],dtype=np.int64)
    for k in range(Nint): # loop for all interactions
        isite1 = list_isite1[k]
        isite2 = list_isite2[k]
        is1 = 1<<isite1
        is2 = 1<<isite2
        is12 = is1 + is2
        wght = 2.0*Jxx[k]
        diag = Jzz[k]
## calculate elements of
## H_loc = Jzz sgmz.sgmz + Jxx (sgmx.sgmx + sgmy.sgmy)
##       = Jzz sgmz.sgmz + 2*Jxx (S+.S- + S-.S+)
        for i in range(Nhilbert): # loop for all spin configurations with fixed Sz
            ii = list_1[i]
            ibit = ii & is12 # find sgmz.sgmz|uu> = |uu> or sgmz.sgmz|dd> = |dd>
            loc[Nint*Nhilbert+i] = i # store diag index
            if (ibit==0 or ibit==is12): # if (spin1,spin2) = (00) or (11): sgmz.sgmz only
                elemnt[Nint*Nhilbert+i] += diag # store +Jzz
#                print("# diag k(interactions) i(Hilbert)",k,i)
#                print("# diag ii  ",np.binary_repr(ii,width=N))
#                print("# diag is12",np.binary_repr(is12,width=N))
#                print("# diag ibit",np.binary_repr(ibit,width=N))
            else: # if (spin1,spin2) = (01) or (10): sgmz.sgmz and (S+.S- or S-.S+)
                elemnt[Nint*Nhilbert+i] -= diag # store -Jzz
                iexchg = ii ^ is12 # find S+.S-|du> = |ud> or S-.S+|ud> = |du>
                newcfg = get_ja_plus_jb(iexchg,irght,ilft,ihfbit,list_ja,list_jb)
                elemnt[k*Nhilbert+i] = wght # store 2*Jxx
                loc[k*Nhilbert+i] = newcfg # store offdiag index
#                print("# offdiag k(interactions) i(Hilbert)",k,i)
#                print("# offdiag ii  ",np.binary_repr(ii,width=N))
#                print("# offdiag is12",np.binary_repr(is12,width=N))
#                print("# offdiag iexc",np.binary_repr(iexchg,width=N))
#    HamCSR = scipy.sparse.csr_matrix((elemnt,(listki,loc)),shape=(Nhilbert,Nhilbert))
#    return HamCSR
    return elemnt, listki, loc

In [9]:
def make_hamiltonian(Nhilbert,elemnt,listki,loc):
    return scipy.sparse.csr_matrix((elemnt,(listki,loc)),shape=(Nhilbert,Nhilbert),dtype=np.float64)

In [10]:
@jit(nopython=True)
def make_lattice(N,J1,J2):
    Jxx = []
    Jzz = []
    list_isite1 = []
    list_isite2 = []
    Nint = 0
    for i in range(N):
        site1 = i
        site2 = (i+1)%N
        site3 = (i+2)%N
#
        list_isite1.append(site1)
        list_isite2.append(site2)
#        Jxx.append(J1)
        Jxx.append(-J1)
        Jzz.append(J1)
        Nint += 1
#
        list_isite1.append(site1)
        list_isite2.append(site3)
#        Jxx.append(J2)
        Jxx.append(-J2)
        Jzz.append(J2)
        Nint += 1
    return Jxx, Jzz, list_isite1, list_isite2, Nint

In [11]:
def main():
    N = 8
    Sz = 0
    J1 = 1.0
    J2 = 0.0

    print("# make list")
    start = time.time()
    Nup, Nhilbert, ihfbit, irght, ilft, iup = init_parameters(N,Sz)
    binirght = np.binary_repr(irght,width=N)
    binilft = np.binary_repr(ilft,width=N)
    biniup = np.binary_repr(iup,width=N)
    print("J1=",J1)
    print("J2=",J2)
    print("N=",N)
    print("Sz=",Sz)
    print("Nup=",Nup)
    print("Nhilbert=",Nhilbert)
    print("ihfbit=",ihfbit)
    print("irght,binirght=",irght,binirght)
    print("ilft,binilft=",ilft,binilft)
    print("iup,biniup=",iup,biniup)
    list_1, list_ja, list_jb = make_list(N,Nup,Nhilbert,ihfbit,irght,ilft,iup)
    end = time.time()
#    print("list_1=",list_1)
#    print("list_ja=",list_ja)
#    print("list_jb=",list_jb)
#    print("")
#    print("i ii binii ja+jb")
#    for i in range(Nhilbert):
#        ii = list_1[i]
#        binii = np.binary_repr(ii,width=N)
#        ind = get_ja_plus_jb(ii,irght,ilft,ihfbit,list_ja,list_jb)
#        print(i,ii,binii,ind)
    print("# time=",end - start)
    print("")

    print("# make lattice")
    start = time.time()
    Jxx, Jzz, list_isite1, list_isite2, Nint = make_lattice(N,J1,J2)
    Jxx = np.array(Jxx)
    Jzz = np.array(Jzz)
    list_isite1 = np.array(list_isite1)
    list_isite2 = np.array(list_isite2)
    print(Jxx)
    print(Jzz)
    print(list_isite1)
    print(list_isite2)
    print("Nint=",Nint)
    end = time.time()
    print("# time=",end - start)
    print("")

    print("# make Hamiltonian")
    start = time.time()
#    HamCSR = make_hamiltonian(Jxx,Jzz,list_isite1,list_isite2,N,Nint,Nhilbert,irght,ilft,ihfbit,list_1,list_ja,list_jb)
    elemnt, listki, loc = make_hamiltonian_child(Jxx,Jzz,list_isite1,list_isite2,N,Nint,Nhilbert,irght,ilft,ihfbit,list_1,list_ja,list_jb)
    HamCSR = make_hamiltonian(Nhilbert,elemnt,listki,loc)
#    print(HamCSR)
    end = time.time()
    print("# time=",end - start)
    print("")

    print("# diagonalization")
    start = time.time()
#    ene,vec = scipy.sparse.linalg.eigsh(HamCSR,k=5)
    ene,vec = scipy.sparse.linalg.eigsh(HamCSR,which="SA",k=2)
    end = time.time()
    #print("# GS energy:",ene[0])
    print("# energy:",ene[0],ene[1])
    vec_sgn = np.sign(np.amax(vec[:,0]))
    print("# GS wave function:")
    for i in range (Nhilbert):
        ii = list_1[i]
        binii = np.binary_repr(ii,width=N)
        print(i,binii,vec[i,0]*vec_sgn)
#    np.savetxt("ref_exact_ground_state.txt",np.array([np.arange(Nhilbert),vec[:,0]*vec_sgn]).T,
    np.savetxt("ref_exact_ground_state.txt",vec[:,0]*vec_sgn,
        header="energy:"+"{}".format(ene[0]))
#
    print("# time=",end - start)
    print("")

In [12]:
main()

# make list
J1= 1.0
J2= 0.0
N= 8
Sz= 0
Nup= 4
Nhilbert= 70
ihfbit= 16
irght,binirght= 15 00001111
ilft,binilft= 240 11110000
iup,biniup= 15 00001111
# time= 1.6862986087799072

# make lattice
[-1. -0. -1. -0. -1. -0. -1. -0. -1. -0. -1. -0. -1. -0. -1. -0.]
[1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0.]
[0 0 1 1 2 2 3 3 4 4 5 5 6 6 7 7]
[1 2 2 3 3 4 4 5 5 6 6 7 7 0 0 1]
Nint= 16
# time= 0.7294869422912598

# make Hamiltonian
# time= 2.370347261428833

# diagonalization
# energy: -14.604373635748699 -12.513676255378307
# GS wave function:
0 00001111 0.007469660603699238
1 00010111 0.03474208920086298
2 00011011 0.05454485719432772
3 00011101 0.03474208920086297
4 00011110 0.007469660603699196
5 00100111 0.054544857194327674
6 00101011 0.16440627939276653
7 00101101 0.13713385079560225
8 00101110 0.03474208920086295
9 00110011 0.09005865420497391
10 00110101 0.16440627939276586
11 00110110 0.05454485719432762
12 00111001 0.05454485719432763
13 00111010 0.03474208920086292
14 00111100 